# LLM tooling for Chemistry

<p style="font-style: italic; font-size: 18px; color: dimgray;">ChatGPT and Gemini</p>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
September 2025

-----

## Introduction

This notebook shows how to do utilize Large Language Model (LLM) [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) with the Raku package ["LLM::Functions"](https://github.com/antononcube/Raku-LLM-Functions), [AAp1].

Non-trivial Stoichiometry examples are demonstrated with tools of the Raku package ["Chemistry::Stoichiometry"](https://raku.land/zef:antononcube/Chemistry::Stoichiometry), [AAp4].

At this point "LLM::Functions" supports function calling in the styles of OpenAI's ChatGPT and Google's Gemini. If the LLM configuration is not using ChatGPT or Gemini, the function calling style used is that of ChatGPT. (Many LLM providers -- other than OpenAI and Gemini -- tend to adhere to OpenAI's API.)

-----

## Setup

Load packages:

In [16]:
use JSON::Fast;
use LLM::Functions;
use LLM::Tooling;
use Chemistry::Stoichiometry;


Define LLM access configurations:

In [17]:
sink my $conf5-mini = llm-configuration('ChatGPT', model => 'gpt-5-mini');
sink my $conf41-mini = llm-configuration('ChatGPT', model => 'gpt-4.1-mini', :8192max-tokens, temperature => 0.4);
sink my $conf-gemini-flash = llm-configuration('Gemini', model => 'gemini-2.0-flash', :8192max-tokens, temperature => 0.4);

### JavaScript::D3

In [18]:
#%javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

In [19]:
#%js
js-d3-list-line-plot(10.rand xx 30, background => 'none')

----

## Diagram

Here is a [Mermaid-JS](https://mermaid.js.org) diagram that shows _single-pass_ LLM-and-tool interaction:


```mermaid
sequenceDiagram
    participant Developer
    participant Model as LLM Service (Gemini)
    participant Tool as External Function

    Note over Developer, Model: 1. Tool Definitions + Messages
    Developer->>Model: molecular-mass(molecule) <br> "What's the mass of SO2?"

    Note over Model: 2. Tool Calls
    Model->>Tool: molecular-mass("SO2")

    Note over Tool: 3. Execute Function Code
    Tool->>Model: {"mass": 64.058}

    Note over Model: 4. Results
    Model->>Developer: {"mass": 64.058}

    Note over Developer: 5. Final Response
    Developer->>Model: The molecular mass of SO2 is 64.058.
```


**Remark:** Instead of one invocation of a local tool -- as shown in the diagram -- often enough multiple LLM-&-local-computer interactions are used for LLM-tool computation.


---

## Chemistry computations examples

The package ["Chemistry::Stoichiometry"](https://raku.land/zef:antononcube/Chemistry::Stoichiometry), [AAp4], provides element data, a grammar (or parser) for chemical formulas, and subs for computing molecular masses and balancing equations. Here is an example of calling `molecular-mass`:

In [20]:
molecular-mass("SO2")

64.058

Balance chemical equation:

In [21]:
'Al + O2 -> Al2O3'
==> balance-chemical-equation

[4*Al + 3*O2 -> 2*Al2O3]

----

## Define package functions as tools

Define a few subs -- _tools_ -- with sub- and argument descriptions (i.e. attached Pod values, or [declarator blocks](https://docs.raku.org/language/pod#Declarator_blocks)):

In [22]:
sink my @tool-objects =
        LLM::Tool.new(&molecular-mass),
        LLM::Tool.new(&balance-chemical-equation)
        ;

Undefined type of parameter ⎡$spec⎦; continue assuming it is a string.


-----

## LLM-tool invocation example **1**

A single prompt or an array of prompts:

In [23]:
sink my $input = "What are the masses of SO2, O3, and C2H5OH? Also balance: C2H5OH + O2 = H2O + CO2";

LLM result:

In [24]:
my $conf = llm-configuration($conf41-mini, tools => @tool-objects);

llm-synthesize(
        [$input, llm-prompt('NothingElse')('JSON')],
        e => $conf, 
        form => sub-parser('JSON'):drop)

{balanced_equation => 1*C2H5OH + 3*O2 -> 2*CO2 + 3*H2O, masses => {C2H5OH => 46.069, O3 => 47.997, SO2 => 64.058}}

**Remark:** It order to see the LLM-tool interaction use the Boolean option (adverb) `:echo` of `llm-synthesize`.

--- 

## "Thoughtful" response

Here is a very informative, "thoughtful" response for a quantitative Chemistry question:

In [25]:
#% markdown
my $input = "How many molecules a kilogram of water has? Use LaTeX for the formulas. (If any.)";

llm-synthesize($input, e => $conf)
==> { .subst(/'\[' | '\]'/, '$$', :g).subst(/'\(' | '\)'/, '$', :g) }() # Make sure LaTeX code has proper fences

To find the number of molecules in a kilogram of water, we need to follow these steps:

1. Calculate the molar mass of water (H₂O).
2. Convert the mass of water (1 kg) to moles.
3. Use Avogadro's number to find the number of molecules.

Step 1: Calculate the molar mass of water.
Water (H₂O) consists of 2 hydrogen atoms and 1 oxygen atom.
- Atomic mass of hydrogen (H) ≈ 1 g/mol
- Atomic mass of oxygen (O) ≈ 16 g/mol

Molar mass of water:
$$ M = 2 \times 1 + 16 = 18 \, \text{g/mol} $$

Step 2: Convert 1 kg of water to grams:
$$ 1 \, \text{kg} = 1000 \, \text{g} $$

Number of moles in 1 kg of water:
$$ n = \frac{1000 \, \text{g}}{18 \, \text{g/mol}} \approx 55.56 \, \text{mol} $$

Step 3: Use Avogadro's number to find the number of molecules.
Avogadro's number $ N_A $ is approximately $ 6.022 \times 10^{23} $ molecules/mol.

Number of molecules:
$$ N = n \times N_A = 55.56 \times 6.022 \times 10^{23} \approx 3.34 \times 10^{25} $$

So, there are approximately $ 3.34 \times 10^{25} $ molecules in 1 kilogram of water.

-----

## References

### Articles, blog posts

[AA1] Anton Antonov,
["LLM function calling workflows (Part 1, OpenAI)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-1-openai/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA2] Anton Antonov,
["LLM function calling workflows (Part 2, Google's Gemini)"](https://rakuforprediction.wordpress.com/2025/06/01/llm-function-calling-workflows-part-2-google-gemini/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[AA3] Anton Antonov,
["LLM function calling workflows (Part 3, Facilitation)"](https://rakuforprediction.wordpress.com/2025/06/08/llm-function-calling-workflows-part-3-facilitation/),
(2025),
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com).

[Gem1] Google Gemini,
["Gemini Developer API"](https://ai.google.dev/gemini-api/docs).

[WRI1] Wolfram Research, Inc.
["LLM-Related Functionality" guide](https://reference.wolfram.com/language/guide/LLMFunctions.html).



### Packages 

[AAp1] Anton Antonov,
[LLM::Functions, Raku package](https://github.com/antononcube/Raku-LLM-Functions),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp2] Anton Antonov,
[WWW::OpenAI, Raku package](https://github.com/antononcube/Raku-WWW-OpenAI),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp3] Anton Antonov,
[WWW::Gemini, Raku package](https://github.com/antononcube/Raku-WWW-Gemini),
(2023-2025),
[GitHub/antononcube](https://github.com/antononcube).

[AAp4] Anton Antonov,
[Chemistry::Stoichiometry, Raku package](https://github.com/antononcube/Raku-Chemistry-Stoichiometry),
(2021-2025),
[GitHub/antononcube](https://github.com/antononcube).